# Import Statements

In [23]:
from vtk import *
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import collections
import plotly.graph_objects as go
import ipywidgets as widgets

# Read, extract data and rounding off

In [24]:
reader = vtkXMLImageDataReader()
reader.SetFileName('data/mixture.vti')
reader.Update()
data = reader.GetOutput()
numPoints = data.GetNumberOfPoints()
listx=[]
listy=[]
listz=[]
isovalue=0.00
dataArr = data.GetPointData().GetArray('ImageFile')
listv=[]
for i in range(numPoints):
  x= data.GetPoint(i)[0]
  y= data.GetPoint(i)[1]
  z= data.GetPoint(i)[2]
  value = dataArr.GetTuple1(i)
  listx.append(x)
  listy.append(y)
  listz.append(z)
  listv.append(value)
listrv = [round(elem, 2) for elem in listv]

# Making and positioning subplots

In [ ]:
fig = make_subplots(rows=1, cols=2,specs=[[{"type": "isosurface"},{"type": "bar"}]])
fig.add_trace(go.Isosurface(
    x=listx, 
    y=listy, 
    z=listz, 
    value=listv, 
    isomin=isovalue, 
    isomax=isovalue,
    colorscale='Plasma',
    showscale=False,
    cmin=min(listv),
    cmax=max(listv)),
    row=1, col=1)
fig.add_trace(go.Histogram(
    x=listrv,
    xbins=dict( # bins used for histogram
      start=-1,
      size=0.05)),
    row=1, col=2) 
fig.update_layout(
    xaxis_title_text='Vortex scalar value', # xaxis label
    yaxis_title_text='Frequency' # yaxis label
) 

# Define widgets

In [26]:
import ipywidgets as widgets
slider= widgets.FloatSlider(
    value=0.00,
    min=min(listrv),
    max=max(listrv),
    step=0.01,
    description='Isoval'
)

reset= widgets.Button(
    description='Reset',
    disabled=False,
    button_style='',
    tooltip='Reset'
)
widgetcontainer=widgets.HBox([slider, reset])
g = go.FigureWidget(fig)

# Handling widget response and display

In [ ]:
def change(x):
  update= round(x['new'], 2)
  updated_range= [i for i in listrv if update-0.25 <= i <= update+0.25]
  with g.batch_update():
    g.data[0].isomin=update
    g.data[0].isomax=update
    g.data[1].x=updated_range
    g.data[1].xbins=dict( 
      start=min(updated_range),
      size=0.01)

def resetfunction(x):
  slider.value=0.00
  with g.batch_update():
    g.data[0].isomin=0.0
    g.data[0].isomax=0.0
    g.data[1].x=listrv
    g.data[1].xbins=dict(start=-1,size=0.05)

slider.observe(change, names="value")       
reset.on_click(resetfunction)
totalcontainer= widgets.VBox([widgetcontainer, g])
display(totalcontainer)